# AR(1) + GARCH(1,1) Model

## Load Packages

In [1]:
using Optim

include("jlFiles/printmat.jl")

println4Ps

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

## Loading Data

In [3]:
xx  = readdlm("Data/FFdSizePs.csv",',',header=true)
xx  = xx[1]
ymd = xx[:,1]     #[YearMonthDay, eg. 20121231]
R   = xx[:,2]     #returns for the smallest size portfolio
xx  = nothing

y = R[2:end]                           #dependent variable, y(t)
x = [ones(size(R,1)-1) R[1:end-1]]     #regressors, [1, y(t-1)]

println()

## The Likelihood Function

Consider a regression equation, where the residual follows a GARCH(1,1) process

$$
\begin{align*}
y_{t}           &  =x_{t}^{\prime}b+u_{t} \ \text{ with }\ u_{t}=v_{t}\sigma_{t}\\
\sigma_{t}^{2}  &  =\omega+\alpha u_{t-1}^{2}+\beta\sigma_{t-1}^{2}.
\end{align*}
$$
Note that we require $(\omega,\alpha,\beta)$ to all be positive and $\alpha + \beta < 1$.

If $v_{t}\sim N(0,1)$, then the likelihood function is

$$
\ln L=-\frac{T}{2}\ln\left(  2\pi\right)  -\frac{1}{2}\sum_{t=1}
^{T}\ln\sigma_{t}^{2}-\frac{1}{2}\sum_{t=1}^{T}\frac{u_{t}^{2}}{\sigma_{t}
^{2}}.
$$

The likelihood function of a GARCH(1,1) model can be coded as in
*garch11LL*. The first function calculates time-varying  variances and
the likelihood contributions (for each period). The second functions forms the
loss function used in the minimization.

In [4]:
function garch11LL(par::Vector,y,x)

  (T,k) = size(x,1,2)

  b                   = par[1:k]             #mean equation, y = x'*b
  (omega,alpha,beta1) = par[k+1:k+3]         #GARCH(1,1) equation: 
                                             #s2(t) = omega + alpha*u(t-1)^2 + beta1*s2(t-1)        
  yhat = x*b
  u    = y - yhat
  s2_0 = var(u)                                 #var(u,1) gives a matrix, var(u) a scalar

  s2    = zeros(typeof(alpha),T)                 #works also with ForwardDiff 
  s2[1] = omega + alpha*s2_0 + beta1*s2_0        #simple, but slow approach
  for t = 2:T                                    #using filter() is perhaps quicker
    s2[t] = omega + alpha*u[t-1]^2 + beta1*s2[t-1]
  end

  LL    = -(1/2)*log(2*pi) - (1/2)*log.(s2) - (1/2)*(u.^2)./s2
  LL[1] = 0               #effectively skip the first observation

  return LL,s2,yhat

end


function garch11LLLoss(par::Vector,y,x)
    par1            = copy(par)
    par1[end-2:end] = abs.(par1[end-2:end])     #impose non-negativity
    LL,  = garch11LL(par1,y,x)
    Loss = -sum(LL)                             #to minimize: -sum(LL)
    return Loss
end


function garch11LLRLoss(par::Vector,y,x,rho)     #with restriction on alpha+beta1 < 1
    par1            = copy(par)
    par1[end-2:end] = abs.(par1[end-2:end])     #impose non-negativity
    LL, = garch11LL(par1,y,x)
    (alpha,beta1) = par1[end-1:end]   #s2(t) = omega + alpha*u(t-1)^2 + beta1*s2(t-1)
    g = [alpha + beta1 - 1]                     #alpha + beta1 < 1
    Loss = -sum(LL) + rho*sum(max.(0,g).^2)
    return Loss
end

garch11LLRLoss (generic function with 1 method)

## Maximize the Likelihood Function

In [5]:
#mean equation, y = x'b
#GARCH(1,1) equation: s2(t) = omega + alfa*u(t-1)^2 + beta1*s2(t-1)

par0 = [mean(y);0;var(y)*0.05;0.05;0.93]         #initial parameter guess

(loglik,s2,yhat) = garch11LL(par0,y,x)            #just testing the log lik
LL               = garch11LLLoss(par0,y,x)
printlnPs("Value of log-likelihood fn at starting guess: ",-LL)

Value of log-likelihood fn at starting guess:  -9846.688


In [6]:
Sol = optimize(par->garch11LLLoss(par,y,x),par0)   #do MLE by optimization with optimize, minimize -sum(LL)

parHat            = Optim.minimizer(Sol)          #extract the optimal solution 
parHat[end-2:end] = abs.(parHat[end-2:end])       #since the likelihood function uses abs(these values)

LLHat             = garch11LLLoss(parHat,y,x)
printlnPs("Value of log-likelihood fn at estimate: ",-LLHat)

println("\nParameter estimates (b[1],b[2],omega,alpha,beta1): ")
printmat(parHat)

Value of log-likelihood fn at estimate:  -8243.608

Parameter estimates (b[1],b[2],omega,alpha,beta1): 
     0.049
     0.256
     0.014
     0.162
     0.824



## Standard Errors of the Estimates

MLE is typically asymptotically normally distributed
$$
\begin{align*}
\sqrt{T}(\hat{\theta}-\theta)  &  \rightarrow^{d}N(0,V)\text{, where } V=I(\theta)^{-1}\text{ with }\\
I(\theta)  &  =-\operatorname*{E}\frac{\partial^{2}\ln L_t}{\partial\theta\partial\theta^{\prime}}
\end{align*}
$$
where $\ln L_t$  is the contribution of period $t$ to the likelihood function and $I(\theta)$ is the information matrix.

The code below calculates numerical derivatives.  


Alternatively, we can use the outer product of the gradients to calculate the
information matrix as
$$
J(\theta)=\operatorname*{E}\left[  \frac{\partial\ln L_t}{\partial\theta
}\frac{\partial\ln L_t}{\partial\theta^{\prime}}\right]
$$

We could also use the "sandwich" estimator

$$
V=I(\theta)^{-1}J(\theta)I(\theta)^{-1}.
$$

In [7]:
T = size(y,1)                    #finding std(coefs) by inverse of information matrix

Ia         = -ForwardDiff.hessian(par->mean(garch11LL(par,y,x)[1]),parHat)
Ia         = (Ia+Ia')/2              #to guarantee symmetry
vcv        = inv(Ia)/T
std_parHat = sqrt.(diag(vcv))

println("std from Hessian")
printmat(std_parHat)

std from Hessian
     0.006
     0.012
     0.001
     0.009
     0.009



In [8]:
LLgrad = ForwardDiff.jacobian(par->garch11LL(par,y,x)[1],parHat)   #T x length(par) matrix, T gradients
J           = LLgrad'LLgrad/T
vcv         = inv(J)/T
stdb_parHat = sqrt.(diag(vcv))                          #std from gradients

vcv         = inv(Ia) * J * inv(Ia)/T
stdc_parHat = sqrt.(diag(vcv))                          #std from sandwich

printlnPs("\nGARCH parameter estimates and 3 different standard errors
    coef     hessian     gradient  sandwich")
printmat([parHat std_parHat stdb_parHat stdc_parHat])


GARCH parameter estimates and 3 different standard errors
    coef     hessian     gradient  sandwich
     0.049     0.006     0.006     0.007
     0.256     0.012     0.012     0.013
     0.014     0.001     0.001     0.003
     0.162     0.009     0.005     0.020
     0.824     0.009     0.005     0.019



In [9]:
println("\nIterate with harder and harder punishment on α + β < 1 restriction. PERHAPS NOT NEEDED")

options = Optim.Options(show_trace=false,show_every=10)
for rho = 0.0:0.1:1
  Sol = optimize(par->garch11LLRLoss(par,y,x,rho),par0,options)
  par = Optim.minimizer(Sol)
  par[end-3:end-1] = abs.(par[end-3:end-1])
  printmat([rho par'])
  par0 = copy(par)
end


Iterate with harder and harder punishment on α + β < 1 restriction. PERHAPS NOT NEEDED
     0.000     0.049     0.256     0.014     0.162     0.824

     0.100     0.049     0.256     0.014     0.162     0.824

     0.200     0.049     0.256     0.014     0.162     0.824

     0.300     0.049     0.256     0.014     0.162     0.824

     0.400     0.049     0.256     0.014     0.162     0.824

     0.500     0.049     0.256     0.014     0.162     0.824

     0.600     0.049     0.256     0.014     0.162     0.824

     0.700     0.049     0.256     0.014     0.162     0.824

     0.800     0.049     0.256     0.014     0.162     0.824

     0.900     0.049     0.256     0.014     0.162     0.824

     1.000     0.049     0.256     0.014     0.162     0.824



## Value at Risk

In [10]:
(_,s2,ER) = garch11LL(parHat,y,x)
VaR95     = -(ER - 1.64*sqrt.(s2))

CovRatio = mean((-y) .>= VaR95)             #coverage ratio for VaR
printlnPs("\nCoverage ratio for VaR(95%): ",CovRatio)


Coverage ratio for VaR(95%):      0.059


In [11]:
y   = floor.(ymd/10000)                        #converting 20121231 to Date
m   = floor.( (ymd-y*10000)/100 )
d   = ymd - y*10000 - m*100
dN  = Date.(y,m,d)

xTicks = [Date(1990);Date(2000);Date(2010)]     #controlling the tick locations
plot(dN[2:end],VaR95,xticks=Dates.value.(xTicks),legend=false)
title!("1-day VaR (95%)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1990-01-01 
 
 
 2000-01-01 
 
 
 2010-01-01 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 1-day VaR (95%) 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 28.9273,327.264 28.973,323.453 29.1103,337.414 29.1561,334.445 29.2018,339.681 29.2476,338.279 29.2933,343.366 29.4306,342.24 29.4763,347.031 29.5221,346.742 
 29.5679,349.871 29.6136,351.084 29.7509,354.785 29.7966,357.916 29.8424,342.148 29.8881,353.702 29.9339,355.943 30.0712,357.401 30.1169,359.859 30.1627,349.013 
 30.2084,357.301 30.2542,355.011 30.3914,359.158 30.4372,359.331 30.4829,362.512 30.5287,365.857 30.5745,367.126 30.7117,367.196 30.7575,355.359 30.8032,361.566 
 30.849,364.033 30.8947,346.897 31.0778,350.97 31.1235,352.203 31.1693,361.242 31.215,352.645 31.3523,353.083 31.398,345.886 31.4438,357.567 31.4895,356.528 
 31.5353,360.396 31.6726,364.229 31.7183,353.743 31.7641,348.744 31.8098,336.034 31.8556,298.885 31.9928,310.941 32.0386,300.793 32.0844,326.139 32.1301,327.102 
 32.1759,330.876 32.3131,335.676 32.3589,286.807 32.4046,307.628 32.4504,327.127 32.4961,321.268 32.6334,327.297 32.6792,281.042 32.7249,288.665 32.7707,310.303 
 32.8164,167.057 32.9537,229.101 32.9994,223.572 33.0452,235.644 33.091,251.98 33.274,255.724 33.3197,246.136 33.3655,273.876 33.4112,289.614 33.457,299.371 
 33.5943,304.234 33.64,300.233 33.6858,311.197 33.7315,313.643 33.7773,315.158 33.9145,328.933 33.9603,320.147 34.006,332.685 34.0518,328.91 34.0976,333.975 
 34.2348,326.901 34.2806,337.336 34.3263,342.527 34.3721,345.498 34.4178,344.287 34.5551,351.115 34.6009,354.891 34.6466,357.974 34.6924,361.079 34.7381,355.267 
 34.8754,357.11 34.9211,360.2 34.9669,365.008 35.0126,366.341 35.0584,366.108 35.1957,366.966 35.2414,367.91 35.2872,368.076 35.3329,368.431 35.3787,370.833 
 35.5617,369.551 35.6075,365.976 35.6532,366.947 35.699,359.366 35.8362,365.596 35.882,364.825 35.9277,365.817 35.9735,369.272 36.0192,367.247 36.1565,368.484 
 36.2023,368.352 36.248,369.709 36.2938,371.558 36.3395,365.759 36.4768,370.095 36.5225,368.089 36.5683,369.495 36.614,369.597 36.6598,366.788 36.7971,369.825 
 36.8428,372.614 36.8886,373.018 36.9343,372.675 36.9801,367.811 37.1173,370.556 37.1631,356.218 37.2089,364.397 37.2546,367.761 37.4376,368.163 37.4834,365.51 
 37.5291,363.602 37.5749,367.699 37.6206,363.844 37.7579,368.141 37.8037,367.01 37.8494,361.858 37.8952,364.631 37.9409,365.524 38.0782,364.245 38.1239,364.151 
 38.1697,358.035 38.2155,362.97 38.2612,366.32 38.3985,364.168 38.4442,367.771 38.49,369.952 38.5357,369.224 38.5815,360.422 38.7188,365.68 38.7645,361.971 
 38.8103,366.142 38.856,368.343 38.9018,368.094 39.039,365.453 39.0848,367.299 39.1305,357.105 39.1763,363.708 39.2221,363.868 39.3593,360.78 39.4051,338.612 
 39.4508,347.869 39.4966,356.401 39.5423,356.848 39.6796,354.458 39.7254,352.096 39.7711,356.373 39.8169,355.701 39.8626,352.396 40.0456,360.426 40.0914,361.83 
 40.1371,361.738 40.1829,355.23 40.3202,359.777 40.3659,353.086 40.4117,360.466 40.4574,363.017 40.5032,362.122 40.6404,361.648 40.6862,361.02 40.732,363.458 
 40.7777,362.822 40.8235,354.929 40.9607,360.054 41.0065,355.307 41.0522,341.998 41.098,352.992 41.1437,353.443 41.281,323.871 41.3268,295.924 41.3725,324.006 
 41.4183,324.471 41.464,330.94 41.6013,337.126 41.647,339.377 41.6928,335.751 41.7386,343.802 41.7843,343.754 41.9216,348.362 41.9673,353.281 42.0131,350.977 
 42.0588,355.795 42.1046,342.904 42.2419,352.226 42.2876,356.598 42.3334,354.203 42.3791,361.27 42.4249,343.911 42.5621,355.322 42.6079,344.165 42.6536,349.662 
 42.6994,357.424 42.7452,348.675 42.8824,357.336 42.9739,360.725 43.0197,362.701 43.0654,335.258 43.2027,346.27 43.2485,350.702 43.2942,357.434 43.34,357.8 
 43.3857,356.431 43.523,355.969 43.5687,352.747 43.6145,358.381 43.6602,358.871 43.706,361.801 43.8433,354.429 43.889,351.217 43.9348,360.111 44.0